In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import cv2
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions,InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16

In [51]:
train_dir =  r"E:\Workstuff\Deep Learning project\Week 1\train"
val_dir = r"E:\Workstuff\Deep Learning project\Week 1\valid"
test_dir = r"E:\Workstuff\Deep Learning project\Week 1\test"


class_names = sorted(os.listdir(train_dir))
print("Class names:", class_names)

Class names: ['ABBOTTS BABBLER', 'ABBOTTS BOOBY', 'ABYSSINIAN GROUND HORNBILL', 'AFRICAN CROWNED CRANE', 'AFRICAN EMERALD CUCKOO', 'AFRICAN FIREFINCH', 'AFRICAN OYSTER CATCHER', 'AFRICAN PIED HORNBILL', 'AFRICAN PYGMY GOOSE', 'ALBATROSS', 'ALBERTS TOWHEE', 'ALEXANDRINE PARAKEET', 'ALPINE CHOUGH', 'ALTAMIRA YELLOWTHROAT', 'AMERICAN AVOCET', 'AMERICAN BITTERN', 'AMERICAN COOT', 'AMERICAN FLAMINGO', 'AMERICAN GOLDFINCH', 'AMERICAN KESTREL']


In [52]:
img_size=224
batch_size=64

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255)


valid_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_size, img_size),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')


valid_generator = valid_datagen.flow_from_directory(val_dir,
                                                    target_size=(img_size, img_size),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')


test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(img_size, img_size),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

Found 3208 images belonging to 20 classes.
Found 100 images belonging to 20 classes.
Found 100 images belonging to 20 classes.


In [56]:
from tensorflow.keras.applications import ResNet50

# Load the pre-trained ResNet50 model with base features only
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
  layer.trainable = False

# Add your custom head for your specific task
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(20, activation='softmax')(x)  # Change num_classes as needed

# Create your custom model
model = keras.Model(inputs=base_model.input, outputs=outputs)

In [57]:
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [58]:
history = model.fit(
      train_dg,
      epochs=25,
      validation_data = validation_dg)

Epoch 1/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 196s 2s/step - accuracy: 0.1255 - loss: 4.5455 - val_accuracy: 0.4700 - val_loss: 1.6623
Epoch 2/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 190s 2s/step - accuracy: 0.4284 - loss: 1.9079 - val_accuracy: 0.7100 - val_loss: 0.8457
Epoch 3/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 186s 2s/step - accuracy: 0.6432 - loss: 1.2247 - val_accuracy: 0.8200 - val_loss: 0.4884
Epoch 4/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 181s 2s/step - accuracy: 0.7819 - loss: 0.7867 - val_accuracy: 0.8900 - val_loss: 0.3829
Epoch 5/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 187s 2s/step - accuracy: 0.8235 - loss: 0.6173 - val_accuracy: 0.8700 - val_loss: 0.3642
Epoch 6/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 187s 2s/step - accuracy: 0.8569 - loss: 0.5257 - val_accuracy: 0.9100 - val_loss: 0.2261
Epoch 7/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 187s 2s/step - accuracy: 0.8594 - loss: 0.5015 - val_accuracy: 0.8700 - val_loss: 0.3139
Epoch 8/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 188s 2s/step - accuracy: 0.8583 - loss: 0.4713 - val_accu